In [ ]:
from http.server import SimpleHTTPRequestHandler, HTTPServer
from urllib import request, error
import urllib.parse
import pandas as pd
from pycaret.clustering import setup, predict_model

# Define bad words to detect intrusion
badwords = ['sleep', 'drop', 'uid', 'select', 'waitfor', 'delay', 'union', 'order by', 'group by']

# Function to extract features from the URL path
def extract_features(path):
    path = urllib.parse.unquote(path)
    badwords_count = sum(path.count(word) for word in badwords)
    single_q = path.count("'")
    double_q = path.count("\"")
    dashes = path.count("--")
    braces = path.count("(")
    spaces = path.count(" ")
    return [single_q, double_q, dashes, braces, spaces, badwords_count]

# Load dataset for clustering
try:
    http = pd.read_csv(r'C:\Users\21070\OneDrive\Desktop\burpy-master\all.csv')
except FileNotFoundError:
    print("File not found. Please check the file path.")
    exit()

# Setup the clustering experiment
try:
    clu1 = setup(data=http, normalize=True, numeric_features=['single_q', 'double_q', 'dashes', 'braces', 'spaces', 'badwords'])
except Exception as e:
    print("An error occurred during setup:", e)
    exit()

# Create the clustering model
try:
    kmeans = create_model('kmeans', num_clusters=2)
except Exception as e:
    print("An error occurred during model creation:", e)
    exit()

# Function to handle HTTP GET requests
def do_GET(self):
    parts = self.path.split('/')
    live_data = extract_features(parts[3])
    result = predict_model(kmeans, data=pd.DataFrame([live_data], columns=['single_q', 'double_q', 'dashes', 'braces', 'spaces', 'badwords']))
    if result['Cluster'][0] == "Cluster 1":
        print("Intrusion Detected")
    super().do_GET()

# Function to handle HTTP POST requests
def do_POST(self):
    print('[1,0,0,0,1]')
    print("Intrusion Detected")

# Custom HTTP proxy handler class
class SimpleHTTPProxy(SimpleHTTPRequestHandler):
    do_GET = do_GET  # Assign the external do_GET function to the class
    do_POST = do_POST  # Assign the external do_POST function to the class

    # Proxy routes
    proxy_routes = {'proxy_route': 'http://demo.testfire.net/'}

    # Function to handle proxy requests
    def proxy_request(self, url):
        try:
            response = request.urlopen(url)
        except error.HTTPError as e:
            print('Error:', e)
            self.send_response_only(e.code)
            self.end_headers()
            return
        self.send_response_only(response.status)
        for name, value in response.headers.items():
            self.send_header(name, value)
        self.end_headers()
        self.copyfile(response, self.wfile)

# Function to start the HTTP server
def start_server():
    with HTTPServer(('127.0.0.1', 8000), SimpleHTTPProxy) as httpd:
        host, port = httpd.socket.getsockname()
        print(f'Listening on http://{host}:{port}')
        httpd.serve_forever()

# Start the HTTP server
start_server()